У нас есть два неполных источника данных об монетизации. При чем один из них ранее выкладывался еще и с ошибками и требует замены.  Нужно как можно более полно и точно восстановать данные за предыдущие периуды, и подготовить код для functions, который будет восстанавливать данные на постоянку, пока новая выгрузка не запуститься. 

In [7]:
import boto3
import os
from config import ACCESS_KEY, SECRET_KEY, TOKEN
import datetime
import pytz

import pandas as pd
pd.set_option('display.max_colwidth', None)

In [8]:
token = TOKEN
headers={'Authorization':token,'Accept':'application/json'}

#'b1gb310irjlk6b99e14g' - аналитика
#'b1gc7vi2ckqausoc5dr7' - спутник

FOLDER_ID = 'b1gc7vi2ckqausoc5dr7' # id каталога из которого береться запрос
ACCESS_KEY = ACCESS_KEY #aws_access_key_id для S3
SECRET_KEY = SECRET_KEY #aws_secret_access_key в s3


In [9]:
bucket_name = 'dwh-asgard' # имя бакета

In [10]:
#создаем подключение к yndex storage через .session.Session()

def get_s3_instance():
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )

s3 = get_s3_instance()

In [11]:
path_on_pc = 'C:/Users/Boris/Desktop/s3' 
date = '2024-03-22'
path = datetime.datetime.strptime(date,'%Y-%m-%d').strftime('year=%Y/month=%m/%d.csv')

entries_df = pd.read_csv(f'{path_on_pc}/entries/{path}')
entries_st_mobile_df = pd.read_csv(f'{path_on_pc}/entries_st_mobile/{path}')

In [74]:
entries_df['monetization'] = entries_df['subsc_restrict'].apply(lambda x: "True" if x == 'freemonetization' else "False")
entries_df['r_address_uuid'] = entries_df['address_uuid']

entries_st_mobile_df = pd.read_csv(f'{path_on_pc}/entries_st_mobile/{path}')
entries_st_mobile_df['l_address_uuid'] = entries_st_mobile_df['address_uuid']

merget_df = entries_st_mobile_df.merge(
        entries_df[['r_address_uuid','address','monetization','partner_uuid']],
        left_on='l_address_uuid', 
        right_on='r_address_uuid',
        how='outer',
        suffixes=('_left', '_right'))


In [75]:
#Находим подъезды без домофонов из entries_st_mobile, которых нет в entries
#Удоляем дубликаты и переменовываем

merget_df_unique_from_entries_st_mobile = merget_df[merget_df['r_address_uuid'] \
    .isnull()][['report_date','address_uuid','partner_uuid_left','monetization_left']]
merget_df_unique_from_entries_st_mobile = merget_df_unique_from_entries_st_mobile \
    .rename(columns={'partner_uuid_left':'partner_uuid','partner_uuid_left':'partner_uuid'})

In [76]:
#оставляем для повтороного соединения только определенные строки без дубликатов из исходника 

entries_df_for_concatenation = entries_df[['report_date','address_uuid','partner_uuid','monetization']]
entries_df_for_concatenation = entries_df_for_concatenation.drop_duplicates()

In [80]:
# соединяем и проверяем на дубликаты
df_concated = pd.concat([entries_df_for_concatenation, merget_df_unique_from_entries_st_mobile]).reset_index().drop('index', axis=1)
df_concated['address_uuid'].value_counts().max()

1

In [99]:
# Создаем вспосогательнаый датафрейм

start_date = datetime.datetime.strptime('2023-04-28','%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2024-02-22','%Y-%m-%d').date()
dates_pd = pd.DataFrame({
        'date_range': pd.date_range(start=start_date, end=end_date),
        'date_key': pd.date_range(start=start_date, end=end_date).strftime('year=%Y/month=%m/%d.csv'),
        'date_key_folder': pd.date_range(start=start_date, end=end_date).strftime('year=%Y/month=%m')
        })

In [101]:
path_on_pc = 'C:/Users/Boris/Desktop/s3' 


for i in range(0,dates_pd.shape[0]):
    key = dates_pd.loc[i,['date_key']].values[0]
    key_folder = dates_pd.loc[i,['date_key_folder']].values[0]

    # переделываем данные из entryes для всех предыдущих дат и добавляем туда наиболее близкую по дате инфу из entryes_st_mobile
    if os.path.isfile(f'{path_on_pc}/new_entryes_st_mobile/{key}') is False:
        entries_df = pd.read_csv(f'{path_on_pc}/entries/{key}')
        entries_df['monetization'] = entries_df['subsc_restrict'].apply(lambda x: "True" if x == 'freemonetization' else "False")
        entries_df_for_concatenation = entries_df[['report_date','address_uuid','partner_uuid','monetization']]
        entries_df_for_concatenation = entries_df_for_concatenation.drop_duplicates()
        df_concated = pd.concat([entries_df_for_concatenation, merget_df_unique_from_entries_st_mobile]).reset_index().drop('index', axis=1)
        os.makedirs(f'{path_on_pc}/new_entryes_st_mobile/{key_folder}', exist_ok=True)
        df_concated.to_csv(f'{path_on_pc}/new_entryes_st_mobile/{key}',sep=',', index=False)
        

In [13]:
# создаем новый вспомогетельный датасет для обновления соединения относительно верных данных

start_date = datetime.datetime.strptime('2024-04-10','%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2024-04-16','%Y-%m-%d').date()
dates_pd = pd.DataFrame({
        'date_range': pd.date_range(start=start_date, end=end_date),
        'date_key': pd.date_range(start=start_date, end=end_date).strftime('year=%Y/month=%m/%d.csv'),
        'date_key_folder': pd.date_range(start=start_date, end=end_date).strftime('year=%Y/month=%m')
        })


path_on_pc = 'C:/Users/Boris/Desktop/s3' 

    # переделываем данные из entryes после 2024-02-23 и добавляем туда наиболее близкую по дате инфу из entryes_st_mobile

for i in range(0,dates_pd.shape[0]):
    key = dates_pd.loc[i,['date_key']].values[0]
    key_folder = dates_pd.loc[i,['date_key_folder']].values[0]
    if os.path.isfile(f'{path_on_pc}/new_entryes_st_mobile/{key}') is False:
        entries_df = pd.read_csv(f'{path_on_pc}/entries/{key}')
        entries_df['monetization'] = entries_df['subsc_restrict'].apply(lambda x: "True" if x == 'freemonetization' else "False")
        entries_df['r_address_uuid'] = entries_df['address_uuid']
        
        entries_st_mobile_df = pd.read_csv(f'{path_on_pc}/entries_st_mobile/{path}')
        entries_st_mobile_df['l_address_uuid'] = entries_st_mobile_df['address_uuid']

        merget_df = entries_st_mobile_df.merge(
            entries_df[['r_address_uuid','address','monetization','partner_uuid']],
            left_on='l_address_uuid', 
            right_on='r_address_uuid',
            how='outer',
            suffixes=('_left', '_right'))
        
        
        merget_df_unique_from_entries_st_mobile = merget_df[merget_df['r_address_uuid'] \
            .isnull()][['report_date','address_uuid','partner_uuid_left','monetization_left']]
        merget_df_unique_from_entries_st_mobile = merget_df_unique_from_entries_st_mobile \
            .rename(columns={'partner_uuid_left':'partner_uuid','partner_uuid_left':'partner_uuid'})
        
        entries_df_for_concatenation = entries_df[['report_date','address_uuid','partner_uuid','monetization']]
        entries_df_for_concatenation = entries_df_for_concatenation.drop_duplicates()
        df_concated = pd.concat([entries_df_for_concatenation, merget_df_unique_from_entries_st_mobile]).reset_index().drop('index', axis=1)
        os.makedirs(f'{path_on_pc}/new_entryes_st_mobile/{key_folder}', exist_ok=True)
        df_concated.to_csv(f'{path_on_pc}/new_entryes_st_mobile/{key}',sep=',', index=False)

In [ ]:
# проверяем на дубликаты
df_concated['address_uuid'].value_counts().max()


In [116]:
# Делаем вспомогательный датасет для загрузки данных

start_date = datetime.datetime.strptime('2024-04-10','%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2024-04-16','%Y-%m-%d').date()
dates_pd = pd.DataFrame({
        'date_range': pd.date_range(start=start_date, end=end_date),
        'date_key': pd.date_range(start=start_date, end=end_date).strftime('year=%Y/month=%m/%d.csv'),
        'date_key_folder': pd.date_range(start=start_date, end=end_date).strftime('year=%Y/month=%m')
        })


In [117]:
dates_pd.loc[i,['date_key']].values[0]

'year=2023/month=06/14.csv'

In [14]:
# загружаем данные

for i in range(0,dates_pd.shape[0]):
    key = dates_pd.loc[i,['date_key']].values[0]
    s3_file_path = f'entries_st_mobile/{key}'
    pc_file_path = f'{path_on_pc}/new_entryes_st_mobile/{key}'
    try:
        s3.upload_file(pc_file_path, bucket_name, s3_file_path)
        print(f'{key} has been uploaded')
    except Exception as e:
        print(e)
        continue
    

year=2024/month=04/10.csv has been uploaded
year=2024/month=04/11.csv has been uploaded
year=2024/month=04/12.csv has been uploaded
year=2024/month=04/13.csv has been uploaded
year=2024/month=04/14.csv has been uploaded
year=2024/month=04/15.csv has been uploaded
year=2024/month=04/16.csv has been uploaded


In [131]:
# Упс, ошибка в имени папки. Копирую внутри бакета в ту папку, в которую нужно

for i in range(0,dates_pd.shape[0]):
    key = dates_pd.loc[i,['date_key']].values[0]
    s3_file_path_old = f'entryes_st_mobile/{key}'
    s3_file_path_new = f'entries_st_mobile/{key}'
    s3.copy_object(
        Bucket=bucket_name,
        CopySource=f'{bucket_name}/{s3_file_path_old}',
        Key=s3_file_path_new
    )